# Gallery Example: M/Hyper/1 Reentrant Queue

This example demonstrates an M/Hyper/1 queueing system with reentrant routing:
- **Arrivals**: Exponential (Poisson process)
- **Service**: Coxian distribution fitted to high SCV
- **Servers**: 1 server
- **Scheduling**: FCFS
- **Reentrant**: Jobs switch class and pass through the queue again

In [ ]:
from line_solver import *
GlobalConstants.set_verbose(VerboseLevel.STD)

In [ ]:
def gallery_mhyp1_reentrant():
    """Create M/Hyper/1-Reentrant queueing model"""
    model = Network('M/Hyper/1-Reentrant')
    
    # Block 1: nodes
    source = Source(model, 'Source')
    queue = Queue(model, 'Queue', SchedStrategy.FCFS)
    sink = Sink(model, 'Sink')
    
    # Block 2: classes
    oclass1 = OpenClass(model, 'Class1')
    oclass2 = OpenClass(model, 'Class2')
    
    # Exponential arrival for Class1, Class2 is disabled at source
    source.set_arrival(oclass1, Exp(1))
    source.set_arrival(oclass2, Disabled())
    
    # Coxian service (high SCV=4) for Class1, Exponential for Class2
    queue.set_service(oclass1, Coxian.fit_mean_and_scv(0.5, 4))
    queue.set_service(oclass2, Exp(3))
    
    # Block 3: topology with class switching (reentrant)
    P = model.init_routing_matrix()
    P.add_route(oclass1, source, queue, 1.0)
    P.add_class_switch(oclass1, oclass2, queue, queue, 1.0)
    P.add_route(oclass2, queue, sink, 1.0)
    model.link(P)
    
    return model

# Create the model
model = gallery_mhyp1_reentrant()

## About High Variability Service

The Coxian distribution with SCV = 4:
- Has high variability in service times
- Some jobs get very quick service, others take much longer
- Models situations with highly variable work content

This high service variability combined with reentrant routing can lead to:
- Longer average queue lengths
- More variable response times
- Higher probability of large backlogs

In [ ]:
# Solve with multiple solvers
print("\n=== Solver Results ===")

# CTMC Solver
solver_ctmc = CTMC(model, cutoff=15)
avg_table_ctmc = solver_ctmc.avg_table()
print("\nCTMC Solver:")
print(avg_table_ctmc)

# Fluid Solver
solver_fluid = FLD(model)
avg_table_fluid = solver_fluid.avg_table()
print("\nFluid Solver:")
print(avg_table_fluid)